In [17]:
import pysolr
from pydantic import BaseModel
from data_loading import load_search_terms, TermType
from models import SearchTerm
from collections import Counter
from typing import Optional, Dict
from tqdm import tqdm
import json
from thefuzz import process

In [10]:
class Match(BaseModel):
    mention: SearchTerm
    match: Optional[Dict]= None
    

In [13]:
with open("/home/tim/MedicalLay/WUMLS/wumls_index_terms_single_valued_deduplicated.json", "r") as fp:
    wumls_terms = json.load(fp)

In [16]:
cui_map = dict()
wumls_entries = []
for wumls_entry in wumls_terms:
    wumls_entries.append(wumls_entry["index_term"])
    cui_map[wumls_entry["index_term"]] = wumls_entry["cui"]

In [18]:
process.extractOne("kolik", wumls_entries)

('kolik', 100)

In [6]:
search_terms = load_search_terms()

In [20]:
matches = []
for search_term in tqdm(search_terms.terms):
    closest_match = None
    score = 0
    for synonym in search_term.stems:
        new_closest_match, new_score = process.extractOne(synonym, wumls_entries)
        if new_score > score:
            closest_match = new_closest_match
            score = new_score
    if  closest_match:
        matches.append(dict(match=closest_match, score=score, cui=cui_map[closest_match]))

  0%|          | 6/7390 [07:05<145:34:01, 70.97s/it] 


KeyboardInterrupt: 